In [85]:
import gensim
import datetime

In [57]:
# Использование другого таггера для вычисления частей речи

from typing import List
from rnnmorph.predictor import RNNMorphPredictor


def morph_words(words: List[str], predictor: RNNMorphPredictor):
    """ 
    Постановка слова в начальную форму и вычисление части речи 
    
    @param predictor: предиктор (RNNMorphPredictor)
    @param word: список строк со словами из предложения
    """
    morphs = predictor.predict(words)
    
    result = []
    for m in morphs:
        result.append((m.normal_form, m.pos))
    
    return result

In [79]:
predictor = RNNMorphPredictor(language="ru")
model = gensim.models.KeyedVectors.load_word2vec_format("../models/180/model.bin", binary=True)

/home/akvarats/dev/ai/ada_datamasters_2019/classification-appeals/.venv/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [75]:
import nltk
import string

def get_clean_rusvectores_words(text: str, predictor: RNNMorphPredictor, word2vec_model):
    """ """
    result = []
    
    for punct in string.punctuation:
        text = text.replace(punct, " ")
    
    morphs = morph_words(nltk.word_tokenize(text, language="russian"), predictor)
    
    for morph in morphs:
        rv_word = "{0}_{1}".format(morph[0], morph[1])
        if rv_word in model.wv:
            result.append(rv_word)
    
    return result

In [95]:
import csv

def read_corpus(file_path):
    """ Читает корпус начального текста """
    result = []

    with open(file_path) as csv_file:
        csv_reader = csv.reader(csv_file)
        next(csv_reader)  # skip header
        for row in csv_reader:
            orig_text = row[1]
            cleaned_rusvectores_words = get_clean_rusvectores_words(orig_text, predictor, model)
            if cleaned_rusvectores_words:
                result.append(
                    dict(
                        id=row[0],
                        orig_text=orig_text,
                        category=row[2],
                        theme=row[3],
                        executor=row[4],
                        cleaned_rusvectores_words=cleaned_rusvectores_words
                    )
                )

    return result

t = datetime.datetime.now()
corpus = read_corpus("../input/NashDomRyazan-29-03-2019.csv")
print("Corpus loaded: {0:.3f} sec".format((datetime.datetime.now() - t).total_seconds()))

/home/akvarats/dev/ai/ada_datamasters_2019/classification-appeals/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  from ipykernel import kernelapp as app


Corpus loaded: 88.472 sec


In [118]:
from sklearn.metrics import pairwise_distances

def get_distance_matrix_for_corpus(corpus):
    """ """
    X = [[i] for i in range(0, len(corpus))]
    
    distance = lambda x, y: model.wmdistance(corpus[int(x[0])]["cleaned_rusvectores_words"], corpus[int(y[0])]["cleaned_rusvectores_words"])
    
    return pairwise_distances(X, metric=distance, n_jobs=24)

In [ ]:
t = datetime.datetime.now()
D = get_distance_matrix_for_corpus(corpus[:-10])
print("Distance matrix: {0:.3f} sec".format((datetime.datetime.now() - t).total_seconds()))

In [115]:
crp = corpus[:-10]

targets = []
categories = dict()  # name -> index
for row in corpus:
    if row["category"] not in categories:
        categories[row["category"]] = len(categories)
    targets.append(categories[row["category"]])

In [116]:
categories

{'Дворовая территория': 0,
 'Парковки': 1,
 'Дороги': 2,
 'Городская территория': 3,
 'Остановки общественного транспорта': 4,
 'Государственное управление и местное самоуправление': 5,
 'Образование (школы, детские сады)': 6,
 'Многоквартирные дома': 7,
 'Бизнес': 8,
 'Природные ресурсы и охрана окружающей среды': 9,
 'ЖКХ': 10,
 'Многофункциональные центры (МФЦ)': 11,
 'Здравоохранение, поликлиники': 12,
 'Трудоустройство': 13,
 'Физическая культура и спорт': 14,
 'Права дольщиков и вопросы переселения': 15,
 'Социальная защита': 16,
 'Общественный транспорт': 17,
 'Культура': 18,
 'Торговля, товары, услуги': 19,
 'Лесное хозяйство': 20,
 'Земельные правоотношения': 21,
 'Общественная безопасность': 22,
 'СМИ': 23,
 'Тестовая категория': 24,
 'Банковская деятельность': 25,
 'Парки': 26,
 'Строительство': 27,
 'Ярмарки выходного дня': 28,
 'Туризм': 29}

In [113]:
from sklearn.neighbors import KNeighborsClassifier

knc = KNeighborsClassifier(n_neighbors=5, metric="precomputed")

knc.fit(D, targets)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='precomputed',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [2]:
text = """Добрый день.
В зимний период 2018-2019 в 5 микрорайоне Дашково-Песочня, в центре которого расположена самая большая школа №75, рассчитанная 1500 учащихся, в ходе выпадения осадков образовались снежные заносы на проезжей части, тротуарах и пешеходных переходах, ведущих к школе, которые мешают движению автомобилей, привозящих и увозящих детей на обучение. А так же идущих детей пешим ходом как одних, так и в сопровождении родителей. Ежедневно в утренние и дневное время в школу направляются в среднем около 5000 человек, для которых затруднено движение в направлении школы и обратно.

Прошу Вас произвести уборку снега с дальнейшим вывозом с проезжей части и тротуаров на улицах:
- Васильевский проезд
- Васильевский переулок
- Васильевская улица
Фото с ежедневной ситуацией в районе школы №75 прилагаю.
https://yadi.sk/d/BE2SvGNigwXG0w
Прилагаю ответ из управления благоустройства города.
"""

test_vectors = get_clean_rusvectores_words(text, predictor, model)

test_distances = [model.wmdistance(test_vectors, c["cleaned_rusvectores_words"]) for c in crp]

result = knc.predict_proba([test_distances])

result

NameError: name 'get_clean_rusvectores_words' is not defined

In [1]:
result[0]

NameError: name 'result' is not defined

In [4]:
import numpy as np
a = np.array([[1, 2, 3, 4, 5]])

In [6]:
a[0][4]

5